In [12]:
## CE 295 - Energy Systems and Control
# Liam Weaver

# 

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
from scipy.integrate import odeint
from scipy import interp
from scipy import signal
import cvxpy as cvx
import pandas as pd
from cvxpy import *
import datetime
np.set_printoptions(threshold=np.inf)



fs = 15;

In [92]:
data=pd.read_csv("NRG_Site_Data.csv")
#pd.options.display.max_seq_items = 2000

#Get Variables from Data
month = np.asarray(data)[:8418,1]
day = np.asarray(data)[:8418,2]
time = np.asarray(data)[:8418,3]
siteload = np.asarray(data)[:8418,4]
pvproduction = np.asarray(data)[:8418,5]
sitemeans = np.asarray(data)[0:24,6]
pvmeans = np.asarray(data)[0:24,7]
sitestdevs = np.asarray(data)[0:24,8]
pvstdevs = np.asarray(data)[0:24,9]
price = np.asarray(data)[0:24,10]
sitevar = np.square(sitestds)
pvvar = np.square(pvstds)
pricereshape = np.reshape(price,(24,1))
pvmeansreshape = np.reshape(pvmeans,(24,1))
sitemeansreshape = np.reshape(sitemeans,(24,1))

pvstds = np.asarray(data)[0:24,9]
sitestds = np.asarray(data)[0:24,8]



In [117]:
#Known Variables
peakload = np.amax(siteload)
Bmax = 0.8*peakload
Emax = Bmax*4 #ie for 4-hour battery
bmax = 1.0 #limit scaling factor to actual Bmax 
bmin = 0.0 #can't be negative
smin = 0.0
smax = 1.5 #would ideally be based on size of roof
#Gmax = 215 #power capacity of grid
Gmax = 1000
effd = 0.9
effc = 0.85
deltat = 1.0
CDF = 1.6449
ssample=1.0
#alpha = 1.282 #CDF for 90%
#alpha = 1.64485 #CDF for 95%
alpha = 2.326 #CDF for 99%
#alpha = 3.09 #CDF for 99.9%
PVC = 360
yearsb = 12.3324   #sweet spot to optimize for battery size
yearss = 25  #Solar lifetime still hits max scaling factor even if bring the years down to 12
cb = 2280.0/(365*yearsb) #$/kW 
#cb = 730.0/(365*years) #$/kW 
cs = 2130.0/(365*yearss) #$/kW   


In [118]:
#Constraints

#Battery Dynamics  (Eq 34)
aeqa1 = np.zeros((24,2))
aeqa2 = effc*deltat*np.eye(24)
aeqa3 = -(1/effd)*deltat*np.eye(24)
aeqa4a = np.eye(24)
aeqa4b = -1*np.eye(24,k=1)
aeqa4 = aeqa4a + aeqa4b
aeqa5a = np.zeros((23,1))
aeqa5b = np.full((1,1),-1)
aeqa5 = np.concatenate((aeqa5a,aeqa5b),axis=0)
aeqa6a = np.full((1,50),0)
aeqa6b = np.full((1,1),1)
aeqa6c = np.full((1,24),0)
aeqa6 = np.concatenate((aeqa6a,aeqa6b,aeqa6c),axis=1)
aeqapre = np.concatenate((aeqa1, aeqa2,aeqa3,aeqa4 ,aeqa5), axis=1)
aeqa = np.vstack((aeqapre,aeqa6))
aeqb = np.zeros((25,24))
aeq = np.hstack((aeqa,aeqb))
Aeq = np.matrix(aeq)

#zeq
zeqa = np.full((25,1),0)
zeq = np.matrix(zeqa)

#Variable inequalities (Eq35,36,39) (will include greaterthan0 in optimization cvx) 
a1a1 = np.eye(75)
a1a2a = np.full((2,1),0)
a1a2b = np.full((48, 1), -1*Bmax)
a1a2c = np.full((25, 1), -1*Emax)
a1a2abc = np.concatenate((a1a2a,a1a2b,a1a2c),axis=0)
a1a2d = np.zeros((75,74))
a1a2 = np.hstack((a1a2abc,a1a2d))
a1a = a1a1 + a1a2
a1b = np.zeros((75,24))
a1 = np.hstack((a1a,a1b))
A = np.matrix(a1)
print np.shape(A)

#'z' here substitutes typical 'b' matrix notation to be clear. Set right side equal to 0 for last 73
z1a1 = np.array((bmax,smax)).reshape((2,1))
z1a2 = np.full((73, 1), 0)
z1a = np.vstack((z1a1,z1a2))
z = np.matrix(z1a)
print np.shape(z)

#C
C1a1 = np.zeros((24,1))
C1a2 = np.reshape(pvmeans, (24, 1))
C1a3 = -1*np.eye(24)
C1a4 = np.eye(24)
C1a5 = np.zeros((24,25))
C1a6 = np.eye(24)
C1a = (1/alpha)*np.concatenate((C1a1,C1a2,C1a3,C1a4,C1a5,C1a6),axis=1)
C = np.matrix(C1a)

#d
d1a = (1/alpha)*np.reshape((-1*sitemeans), (24, 1))
d = np.matrix(d1a)

#Capital cost
ct1a1 = np.full((1,1),cb*Bmax)
ct1b1 = np.full((1,1),cs*PVC)
ct1c = np.zeros((1,97))
ct1 = np.concatenate((ct1a1,ct1b1,ct1c),axis=1)

#Marginal Cost
ct2a = np.zeros((1,75))
ct2c = np.reshape(pricereshape,(1,24))
ct2 = np.concatenate((ct2a,ct2c),axis=1)

#Final Cost matrix
ct = ct1 + ct2


(75, 99)
(75, 1)


In [119]:
x = Variable(99,1)
print x[1]

# Create two constraints.
constraints = [Aeq * x == zeq,
               A*x <= z,
              x >= 0,
              cvx.norm(pvstds[0]*x[1]+sitestds[0]) <= C[0]*x + d[0],  #this is only computing 1 norm, need 48
              cvx.norm(pvstds[1]*x[1]+sitestds[1]) <= C[1]*x + d[1],
               cvx.norm(pvstds[2]*x[1]+sitestds[2]) <= C[2]*x + d[2],
               cvx.norm(pvstds[3]*x[1]+sitestds[3]) <= C[3]*x + d[3],
               cvx.norm(pvstds[4]*x[1]+sitestds[4]) <= C[4]*x + d[4],
               cvx.norm(pvstds[5]*x[1]+sitestds[5]) <= C[5]*x + d[5],
               cvx.norm(pvstds[6]*x[1]+sitestds[6]) <= C[6]*x + d[6],
               cvx.norm(pvstds[7]*x[1]+sitestds[7]) <= C[7]*x + d[7],
               cvx.norm(pvstds[8]*x[1]+sitestds[8]) <= C[8]*x + d[8],
               cvx.norm(pvstds[9]*x[1]+sitestds[9]) <= C[9]*x + d[9],
               cvx.norm(pvstds[10]*x[1]+sitestds[10]) <= C[10]*x + d[10],
               cvx.norm(pvstds[11]*x[1]+sitestds[11]) <= C[11]*x + d[11],
               cvx.norm(pvstds[12]*x[1]+sitestds[12]) <= C[12]*x + d[12],
               cvx.norm(pvstds[13]*x[1]+sitestds[13]) <= C[13]*x + d[13],
               cvx.norm(pvstds[14]*x[1]+sitestds[14]) <= C[14]*x + d[14],
               cvx.norm(pvstds[15]*x[1]+sitestds[15]) <= C[15]*x + d[15],
               cvx.norm(pvstds[16]*x[1]+sitestds[16]) <= C[16]*x + d[16],
               cvx.norm(pvstds[17]*x[1]+sitestds[17]) <= C[17]*x + d[17],
               cvx.norm(pvstds[18]*x[1]+sitestds[18]) <= C[18]*x + d[18],
               cvx.norm(pvstds[19]*x[1]+sitestds[19]) <= C[19]*x + d[19],
               cvx.norm(pvstds[20]*x[1]+sitestds[20]) <= C[20]*x + d[20],
               cvx.norm(pvstds[21]*x[1]+sitestds[21]) <= C[21]*x + d[21],
               cvx.norm(pvstds[22]*x[1]+sitestds[22]) <= C[22]*x + d[22],
               cvx.norm(pvstds[23]*x[1]+sitestds[23]) <= C[23]*x + d[23],

                ]
# Form objective.
obj = Minimize(ct*x)

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
#print "optimal var", x.value,

x1 = x.value[0:2]
x2 = x.value[2:26]
x3 = x.value[26:50]
x4 = x.value[50:75]
x5 = x.value[75:98]
print "b,s:", x1
print "Bc:", x2
print "Bd:", x3
print "E:", x4
print "G:",x5
#print prob

var4652[1, 0]
status: optimal
optimal value 1926.02592494
b,s: [[ 0.47126051]
 [ 1.49999997]]
Bc: [[  5.34656246e+01]
 [  4.19580833e+01]
 [  4.00416500e+01]
 [  4.23221033e+01]
 [  4.89428250e+01]
 [  6.66596571e+01]
 [  9.61066064e+01]
 [  1.31113917e+02]
 [  1.24771769e+02]
 [  1.08102331e+02]
 [  1.11214181e-06]
 [  1.55898438e-06]
 [  1.69774273e-06]
 [  4.58324003e-07]
 [  4.28391251e-07]
 [  4.21142109e-07]
 [  4.17160880e-07]
 [  4.17149059e-07]
 [  4.14914507e-07]
 [  2.13682585e-06]
 [  9.07255520e-07]
 [  5.34666975e-06]
 [  1.97866773e-06]
 [  1.19997683e-06]]
Bd: [[  6.32865159e-07]
 [  6.20134006e-07]
 [  6.39328304e-07]
 [  6.67538801e-07]
 [  7.17840090e-07]
 [  7.91593902e-07]
 [  8.63930747e-07]
 [  9.12239174e-07]
 [  9.64159504e-07]
 [  1.02633760e-06]
 [  2.48935474e-06]
 [  9.45710470e-07]
 [  6.86701070e-07]
 [  1.15211308e+02]
 [  1.04292286e+02]
 [  1.34676039e+02]
 [  8.99181525e+01]
 [  6.30610244e+01]
 [  6.92568761e+01]
 [  6.78345698e-07]
 [  1.59504324e-0

In [74]:
#Without chance constraints
x = Variable(99,1)
print x[1]

# Create two constraints.
constraints = [Aeq * x == zeq,
               A*x <= z,
              x >= 0,
              0 <= C[0]*x + d[0],  #this is only computing 1 norm, need 48
              0 <= C[1]*x + d[1],
               0 <= C[2]*x + d[2],
               0 <= C[3]*x + d[3],
               0 <= C[4]*x + d[4],
               0 <= C[5]*x + d[5],
               0 <= C[6]*x + d[6],
              0 <= C[7]*x + d[7],
               0 <= C[8]*x + d[8],
               0 <= C[9]*x + d[9],
              0 <= C[10]*x + d[10],
              0 <= C[11]*x + d[11],
               0 <= C[12]*x + d[12],
               0 <= C[13]*x + d[13],
              0 <= C[14]*x + d[14],
               0 <= C[15]*x + d[15],
               0 <= C[16]*x + d[16],
               0 <= C[17]*x + d[17],
              0 <= C[18]*x + d[18],
               0 <= C[19]*x + d[19],
               0 <= C[20]*x + d[20],
               0 <= C[21]*x + d[21],
               0 <= C[22]*x + d[22],
               0 <= C[23]*x + d[23],

                ]
# Form objective.
obj = Minimize(ct*x + g)

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
#print "optimal var", x.value,

x1 = x.value[0:2]
x2 = x.value[2:26]
x3 = x.value[26:50]
x4 = x.value[50:75]
x5 = x.value[75:98]
print "b,s:", x1
print "Bc:", x2
print "Bd:", x3
print "E:", x4
print "G:",x5
print "S:",x.value[1:2]*pvmeans
#print prob

var2771[1, 0]
status: optimal
optimal value 2306.56997072
b,s: [[ 0.37845832]
 [ 1.5       ]]
Bc: [[  1.26955157e+01]
 [  6.26341346e+00]
 [  4.56537876e+00]
 [  3.64465585e+00]
 [  3.00674059e+00]
 [  2.48741809e+00]
 [  2.00605658e+00]
 [  1.52122932e+00]
 [  1.09324069e+00]
 [  6.84022336e+01]
 [  1.13970000e+02]
 [  1.28585000e+02]
 [  1.23575000e+02]
 [  9.66350002e+01]
 [  3.66550002e+01]
 [  3.79498404e-08]
 [  2.29373996e-08]
 [  1.13774425e-08]
 [  1.12897828e-08]
 [  3.63568399e-08]
 [  3.58745381e-08]
 [  1.82539872e-07]
 [  9.43304836e-08]
 [  6.25248016e-08]]
Bd: [[  3.70979372e-08]
 [  3.89019460e-08]
 [  4.00540768e-08]
 [  4.08715376e-08]
 [  4.16165262e-08]
 [  4.23564192e-08]
 [  4.33821432e-08]
 [  4.38790478e-08]
 [  3.81052147e-08]
 [  3.84628291e-08]
 [  3.81167625e-08]
 [  1.61236964e-08]
 [  3.55827241e-08]
 [  3.67058523e-08]
 [  3.69175470e-08]
 [  6.25849999e+01]
 [  1.28585000e+02]
 [  1.28585000e+02]
 [  1.28585000e+02]
 [  5.22697751e+00]
 [  9.33902248e+0